In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from typing import List, Tuple

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml

import tqdm
tqdm.tqdm.pandas()
#plt.style.use('ggplot')

In [20]:
df_experiments_1 = seml.get_results('rgnn_victims_cora_citeseer_3',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [23]:
df_experiments_2 = seml.get_results('rgnn_rpprgo_products',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])

In [25]:
df_experiments_2.columns

Index(['_id', 'batch_id', 'slurm.experiments_per_job',
       'slurm.sbatch_options.gres', 'slurm.sbatch_options.mem',
       'slurm.sbatch_options.cpus-per-task', 'slurm.sbatch_options.time',
       'slurm.sbatch_options.nodes', 'slurm.sbatch_options.job-name',
       'slurm.sbatch_options.array', 'slurm.sbatch_options.output',
       'slurm.array_id', 'slurm.task_id', 'config.overwrite',
       'config.db_collection', 'config.dataset', 'config.model_params.label',
       'config.model_params.model', 'config.model_params.dropout',
       'config.model_params.n_filters', 'config.model_params.gdc_params',
       'config.model_params.svd_params', 'config.model_params.batch_norm',
       'config.model_params.jaccard_params',
       'config.model_params.do_cache_adj_prep', 'config.model_params.alpha',
       'config.model_params.eps', 'config.model_params.hidden_size',
       'config.model_params.nlayers', 'config.model_params.ppr_normalization',
       'config.model_params.topk', 'config.

In [ ]:
train_params = {
    "Learning Rate": "lr",
    "Weight Decay": "weight_decay",
    "Max Epochs": "max_epochs",
    "Patience": "patience",
    "Batch Size": "batch_size",
    "Annealing Scheduler": "use_annealing_scheduler",
    "AS with Warm Restart": "scheduler_warm_restarts",
}
train_params = {k: "config.train_params."+v for k, v in train_params.items()}

In [24]:
df_experiments = pd.DataFrame(columns=["label"])

,_id,batch_id,slurm.experiments_per_job,slurm.sbatch_options.gres,slurm.sbatch_options.mem,slurm.sbatch_options.cpus-per-task,slurm.sbatch_options.time,slurm.sbatch_options.nodes,slurm.sbatch_options.job-name,slurm.sbatch_options.array,slurm.sbatch_options.output,slurm.array_id,slurm.task_id,config.overwrite,config.db_collection,config.dataset,config.model_params.label,config.model_params.model,config.model_params.dropout,config.model_params.n_filters,config.model_params.gdc_params,config.model_params.svd_params,config.model_params.batch_norm,config.model_params.jaccard_params,config.model_params.do_cache_adj_prep,config.model_params.alpha,config.model_params.eps,config.model_params.hidden_size,config.model_params.nlayers,config.model_params.ppr_normalization,config.model_params.topk,config.train_params.lr,config.train_params.weight_decay,config.train_params.patience,config.train_params.max_epochs,config.train_params.annealing_scheduler_T_0,config.train_params.batch_mult_val,config.train_params.batch_size,config.train_params.forward_batch_size,config.train_params.log_dir,config.train_params.scheduler_step,config.train_params.scheduler_time,config.train_params.use_annealing_scheduler,config.binary_attr,config.make_undirected,config.make_unweighted,config.seed,config.artifact_dir,config.model_storage_type,config.device,config.display_steps,config.data_dir,config.data_device,result.accuracy,result.trace_val.loss,result.trace_val.acc,result.trace_train.loss,result.trace_train.acc,result.model_path,config.train_params.optim,config.train_params.scheduler_warm_restarts,config.model_params.mean,config.model_params.mean_kwargs.temperature,slurm.sbatch_options.qos
0,1,1,1,gpu:1,32G,4,0-12:00,1,rgnn_papers100M_train_1,0-1,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6301800,0,1,rgnn_rpprgo_products,ogbn-products,Vanilla PPRGo Diffuse Embeddings,PPRGoDiffEmbWrapper,0.00,64,None,None,True,None,True,0.10,0.0001,64,4,row,64,0.0050,0.000000,300,30,10,1,8192,128,runs/products,60,epoch,False,False,True,True,0,cache,products,0,10,/nfs/students/schmidtt/datasets/,cpu,0.645060,"[3.919611930847168, 3.6623246669769287, 3.4342...","[0.0145263671875, 0.100341796875, 0.2802734375...","[3.841193914413452, 3.830347776412964, 3.81822...","[0.31005859375, 0.3309326171875, 0.33666992187...",cache/products/products_2.pt,NaN,NaN,NaN,NaN,NaN
1,2,1,1,gpu:1,32G,4,0-12:00,1,rgnn_papers100M_train_1,0-1,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6301800,1,2,rgnn_rpprgo_products,ogbn-products,Vanilla PPRGo Diffuse Embeddings,PPRGoDiffEmbWrapper,0.00,64,None,None,True,None,True,0.25,0.0001,64,4,row,64,0.0050,0.000000,300,30,10,1,8192,128,runs/products,60,epoch,False,False,True,True,0,cache,products,0,10,/nfs/students/schmidtt/datasets/,cpu,0.637576,"[3.922366142272949, 3.665022373199463, 3.44244...","[0.0147705078125, 0.0948486328125, 0.275878906...","[3.8345861434936523, 3.8164963722229004, 3.797...","[0.279541015625, 0.34521484375, 0.35986328125,...",cache/products/products_1.pt,NaN,NaN,NaN,NaN,NaN
2,5,3,1,gpu:1,32G,4,0-12:00,1,rgnn_papers100M_train_3,0-71,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6301815,0,5,rgnn_rpprgo_products,ogbn-products,Vanilla PPRGo,PPRGo,0.00,64,None,None,True,None,True,0.10,0.0001,64,3,row,64,0.0100,0.000000,300,30,10,1,16384,512,runs/products,60,epoch,False,False,True,True,0,cache,products,0,10,/nfs/students/schmidtt/datasets/,cpu,0.616677,"[3.8298656940460205, 3.8019614219665527, 3.766...","[0.1126708984375, 0.15936279296875, 0.24151611...","[3.861355781555176, 3.7947967052459717, 3.7352...","[0.01824951171875, 0.16986083984375, 0.2332153...",cache/products/products_3.pt,NaN,NaN,NaN,NaN,NaN
3,6,3,1,gpu:1,32G,4,0-12:00,1,rgnn_papers100M_train_3,0-71,/nfs/homedirs/schmidtt/git/robust-gnns-at-scal...,6301815,1,6,rgnn_rpprgo_products,ogbn-products,Vanilla PPRGo,PPRGo,0.00,64,None,None,True,None,True,0.10,0.0001,64,3,row,64,0.0100,0.001000,300,30,10,1,16384,512,runs/products,60,epoch,False,False,True,True,0,cache,products

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [6]:
labels_to_plot = [
    'Vanilla GCN',
    'Vanilla GDC',
    'SVD GCN',
    'Jaccard GCN',
    'RGCN',
    'Soft Medoid GDC (T=0.5)',
    #'Soft Median GDC (T=1.0)',
    #'Soft Median GDC (T=0.5)',
    'Soft Median GDC (T=0.2)'
]

In [7]:
dataset_map = {
    'cora_ml': r'\textbf{Cora ML}', # '\rotatebox{90}{Cora ML~\citep{Bojchevski2018}}',
    'citeseer': r'\textbf{Citeseer}', # ~\citep{McCallum2000}
    'pubmed': r'\textbf{PubMed}', # ~\citep{Sen2008}
    'ogbn-arxiv': r'\textbf{arXiv}', # ~\citep{Hu2020}
    'ogbn-products': r'\textbf{Products}', # ~\citep{Hu2020},
    'ogbn-papers100M': r'\textbf{Papers 100M}'
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [9]:
pm = r'\(\pm\)'
bpm = r'\(\boldsymbol{\pm}\)'

In [16]:
df_experiments.columns

Index(['_id', 'batch_id', 'slurm.experiments_per_job',
       'slurm.sbatch_options.gres', 'slurm.sbatch_options.mem',
       'slurm.sbatch_options.cpus-per-task', 'slurm.sbatch_options.time',
       'slurm.sbatch_options.nodes', 'slurm.sbatch_options.job-name',
       'slurm.sbatch_options.array', 'slurm.sbatch_options.output',
       'slurm.array_id', 'slurm.task_id', 'config.overwrite',
       'config.db_collection', 'config.dataset', 'config.model_params.label',
       'config.model_params.model', 'config.model_params.dropout',
       'config.model_params.n_filters', 'config.model_params.gdc_params',
       'config.model_params.svd_params', 'config.model_params.jaccard_params',
       'config.model_params.do_cache_adj_prep', 'config.model_params.alpha',
       'config.model_params.eps', 'config.model_params.hidden_size',
       'config.model_params.mean', 'config.model_params.mean_kwargs.k',
       'config.model_params.mean_kwargs.temperature',
       'config.model_params.mean_kwar

In [17]:
df_experiments.shape

(90, 61)

In [18]:
df_results = [
    pd.DataFrame(r) 
    for r in df_experiments['result.results']
]
for df_result, (_, df_experiment) in zip(df_results, df_experiments.iterrows()):
    df_result['dataset'] = df_experiment['config.dataset']
    df_result['attack'] = df_experiment['config.attack']
    df_result['seed'] = df_experiment['config.seed']
    df_result['batch_id'] = df_experiment['batch_id']
    df_result['novel_loss'] = (
        (df_experiment['config.attack_params.stop_optimizing_if_label_flipped'] == True)
        | (df_experiment['config.attack_params.loss_type'] == 'MCE')
        | (df_experiment['config.attack_params.loss_type'] == 'tanhCW')
    )

df_results = pd.concat(df_results, ignore_index=True)
df_results = df_results.sort_values('batch_id')
df_results = df_results.drop_duplicates([
    c for c in df_results.columns if c != 'batch_id' and c != 'accuracy'
], keep='last')

df_results

,label,epsilon,accuracy,dataset,attack,seed,batch_id,novel_loss
2253,Vanilla GDC,0.25,0.569445,ogbn-products,PRBCD,0,3,False
2261,Vanilla GDC,0.01,0.667292,ogbn-products,PRBCD,1,3,False
2255,Vanilla GCN,0.25,0.544153,ogbn-products,PRBCD,0,3,True
2257,Vanilla GDC,0.25,0.564142,ogbn-products,PRBCD,0,3,True
2259,Vanilla GCN,0.01,0.684139,ogbn-products,PRBCD,1,3,False
...,...,...,...,...,...,...,...,...
6385,Vanilla GCN,0.10,0.401848,ogbn-arxiv,PRBCD,0,7,False
6386,Vanilla GCN,0.25,0.297266,ogbn-arxiv,PRBCD,0,7,False
6387,Vanilla GCN,0.00,0.702776,ogbn-arxiv,PRBCD,0,7,False
6363,Soft Median GDC (T=5.0),0.01,0.484723,ogbn-arxiv,GreedyRBCD,5,7,True


In [19]:
df_results.attack.unique()

array(['PRBCD', 'GANG', 'GreedyRBCD', 'DICE', 'FGSM', 'PGD'], dtype=object)

In [20]:
df_results.label.unique()

array(['Vanilla GDC', 'Vanilla GCN', 'Soft Median GDC (T=0.5)',
       'Soft Median GDC (T=0.2)', 'Soft Medoid GDC (T=1.0)',
       'Soft Median GDC (T=1.0)', 'SVD GCN', 'Soft Medoid GDC (T=0.2)',
       'Soft Medoid GDC (T=0.5)', 'Jaccard GCN', 'RGCN',
       'Soft Medoid GDC (T=5.0)', 'Soft Median GDC (T=50.0)',
       'Soft Medoid GDC (T=50.0)', 'Soft Median GDC (T=5.0)'],
      dtype=object)

In [31]:
architecture_c = r'\textbf{Architecture}'
dataset_c = r'   '
attack_c = r'\textbf{Attack}'
epsilons_c = r'Frac. edges \(\boldsymbol{\epsilon}\)'

epsilons = [0.01, 0.05, 0.1, 0.25]
epsilon_marks = ['', r'\textit', r'\underline', r'\textbf']

In [32]:
def transform_label(label: str):
    return label.replace('\n', ' ').replace(' (T=0.5)','').replace(' (T=0.2)','')

In [33]:
df = []

for (dataset, attack, label, epsilon), df_group in df_results[
    (df_results['novel_loss']
     | (df_results['attack'] == 'DICE')
     | (df_results['attack'] == 'GANG'))
    & df_results['label'].isin(labels_to_plot)
].groupby(['dataset', 'attack', 'label', 'epsilon']):
    if len(df_group.seed.unique()) != 3:
        print(f'For {dataset}-{attack}-{epsilon} collected runs for seed {df_group.seed.tolist()}')

    accurcy = calc_mean_and_error(df_group.accuracy, df_group.seed, with_error=False)
    df.append({
        dataset_c: dataset_map[dataset],
        architecture_c: transform_label(label),
        attack_c: attack_map[attack],
        epsilons_c: epsilon,
        'accuracy': accurcy
    })
    
df = pd.DataFrame(df)
df

For ogbn-products-GANG-0.0 collected runs for seed [1, 0]
For ogbn-products-GANG-0.01 collected runs for seed [1, 0]
For ogbn-products-GANG-0.05 collected runs for seed [0, 1]
For ogbn-products-GANG-0.1 collected runs for seed [0]
For ogbn-products-GANG-0.25 collected runs for seed [0]
For ogbn-products-GANG-0.0 collected runs for seed [1, 0]
For ogbn-products-GANG-0.01 collected runs for seed [0, 1]
For ogbn-products-GANG-0.05 collected runs for seed [0, 1]
For ogbn-products-GANG-0.1 collected runs for seed [0]
For ogbn-products-GANG-0.25 collected runs for seed [0]


,,\textbf{Architecture},\textbf{Attack},Frac. edges \(\boldsymbol{\epsilon}\),accuracy
0,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.00,0.714
1,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.01,0.712
2,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.05,0.707
3,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.10,0.699
4,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.25,0.676
...,...,...,...,...,...
505,\textbf{Products},Vanilla GDC,\textbf{PR-BCD (ours)},0.00,0.709
506,\textbf{Products},Vanilla GDC,\textbf{PR-BCD (ours)},0.01,0.670
507,\textbf{Products},Vanilla GDC,\textbf{PR-BCD (ours)},0.05,0.627
508,\textbf{Products},Vanilla GDC,\textbf{PR-BCD (ours)},0.10,0.602


In [34]:
df = df[
    ~((df[attack_c] == attack_map['PGD']) & (df[dataset_c] == dataset_map['pubmed']))
    & ~((df[attack_c] == attack_map['FGSM']) & (df[dataset_c] == dataset_map['pubmed']))
    #& ~((df[architecture_c] == transform_label('Soft Medoid GDC (T=0.5)')) & (df[dataset_c] == dataset_map['pubmed']))
]

In [35]:
df_accuracy = df[(df[epsilons_c] == 0) & (df[attack_c] == attack_map['PRBCD'])]
df_accuracy = df_accuracy.drop(columns=[attack_c, epsilons_c])
df_accuracy = df_accuracy.set_index([dataset_c, architecture_c])
df_accuracy.columns = pd.MultiIndex.from_product([['\textbf{Accuracy}'], [' ']])
df_accuracy

\textbf{Accuracy}
                                                         
                  \textbf{Architecture}                  
\textbf{Citeseer} Jaccard GCN                       0.714
                  RGCN                              0.646
                  SVD GCN                           0.641
                  Soft Median GDC                   0.709
                  Soft Medoid GDC                   0.707
                  Vanilla GCN                       0.712
                  Vanilla GDC                       0.709
\textbf{Cora ML}  Jaccard GCN                       0.819
                  RGCN                              0.800
                  SVD GCN                           0.761
                  Soft Median GDC                   0.819
                  Soft Medoid GDC                   0.817
                  Vanilla GCN                       0.825
                  Vanilla GDC                       0.831
\textbf{arXiv}    Soft Medoid GDC                   0.599
                  Vanilla GCN                       0.705
                  Vanilla GDC                       0.617
\textbf{Products} Vanilla GCN                       0.719
                  Vanilla GDC                       0.709

In [36]:
df = df[df[epsilons_c].isin(epsilons)].copy()

In [37]:
from functools import partial

def _mark_best_and_second_best(vector: np.ndarray, 
                               first_mark: str = r'\textbf',
                               second_mark: str = r'\underline',
                               is_higher_better : bool = True) -> np.ndarray:
    vector = vector.astype(object)
    values = np.unique(vector[vector == vector])
    if not len(values):
        return vector
    
    if first_mark:
        if is_higher_better:
            mask = [vector == values[-1]]
        else:
            mask = [vector == values[0]]
        vector[mask] = np.char.add(np.char.add(first_mark + '{', vector[mask]), '}')
    if second_mark:
        if is_higher_better:
            mask = [vector == values[-2]]
        else:
            mask = [vector == values[1]]
        vector[mask] = np.char.add(np.char.add(second_mark + '{', vector[mask]), '}')
    return vector
    

def mark_best_and_second_best(df: pd.DataFrame, 
                              first_mark: str = r'\textbf',
                              second_mark: str = r'\underline',
                              axis: int = 0,
                              is_higher_better : bool = True) -> pd.DataFrame:
    df[:] = np.apply_along_axis(
        partial(_mark_best_and_second_best, first_mark=first_mark, 
                second_mark=second_mark, is_higher_better=is_higher_better),
        axis=axis,
        arr=df.values
    )
    return df

In [38]:
df[architecture_c] = df[architecture_c].astype("category")
df[architecture_c].cat.set_categories(
    [transform_label(l) for l in labels_to_plot],
    inplace=True
)
df[dataset_c] = df[dataset_c].astype("category")
df[dataset_c].cat.set_categories(
    dataset_order,
    inplace=True
)

df_piv = pd.pivot_table(
    df, 
    index=[dataset_c, architecture_c], 
    columns=[attack_c, epsilons_c], #['type', attack_c, ' '], 
    values='accuracy',
    aggfunc=lambda x: ' '.join(x)
)

df_piv = df_piv[[c for attack in attack_order for c in df_piv.columns if c[0] == attack]]

for epsilon, mark in zip(epsilons, epsilon_marks):
    current_columns = [c for c in df_piv.columns if c[1] == epsilon]
    df_piv[current_columns] = df_piv[current_columns].groupby(dataset_c).apply(
        partial(mark_best_and_second_best, first_mark=mark, second_mark='', axis=1, is_higher_better=False)
    )

def acc_to_mean_and_error(group: pd.DataFrame, with_error=True, decimal_places: int = 3):
    return calc_mean_and_error(group['accuracy'], group['seed'], with_error=with_error, decimal_places=decimal_places)

df_piv = df_piv.join(df_accuracy)
df_piv = df_piv.fillna('-')
df_piv

<ipython-input-37-6cca4f51faed>:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  vector[mask] = np.char.add(np.char.add(first_mark + '{', vector[mask]), '}')


\textbf{Attack}                         \textbf{DICE}                       \
Frac. edges \(\boldsymbol{\epsilon}\)            0.01   0.05    0.1   0.25   
                  \textbf{Architecture}                                      
\textbf{Cora ML}  Vanilla GCN                   0.822  0.813  0.803  0.765   
                  Vanilla GDC                   0.829  0.820  0.807  0.774   
                  SVD GCN                       0.758  0.754  0.741  0.696   
                  Jaccard GCN                   0.817  0.810  0.801  0.769   
                  RGCN                          0.799  0.794  0.785  0.756   
                  Soft Medoid GDC               0.816  0.813  0.806  0.793   
                  Soft Median GDC               0.819  0.814  0.811  0.797   
\textbf{Citeseer} Vanilla GCN                   0.710  0.702  0.691  0.663   
                  Vanilla GDC                   0.706  0.694  0.682  0.649   
                  SVD GCN                       0.637  0.625  0.606  0.566   
                  Jaccard GCN                   0.712  0.707  0.699  0.676   
                  RGCN                          0.643  0.634  0.624  0.597   
                  Soft Medoid GDC               0.707  0.703  0.701  0.694   
                  Soft Median GDC               0.709  0.708  0.701  0.693   
\textbf{arXiv}    Vanilla GCN                   0.702  0.678  0.661  0.609   
                  Vanilla GDC                   0.615  0.574  0.538  0.462   
                  Soft Medoid GDC               0.596  0.586  0.576  0.554   
\textbf{Products} Vanilla GCN                       -      -      -      -   
                  Vanilla GDC                       -      -      -      -   

\textbf{Attack}                         \textbf{GANG (ours)}                  \
Frac. edges \(\boldsymbol{\epsilon}\)                   0.01            0.05   
                  \textbf{Architecture}                                        
\textbf{Cora ML}  Vanilla GCN                          0.809           0.766   
                  Vanilla GDC                          0.822           0.788   
                  SVD GCN                              0.770           0.764   
                  Jaccard GCN                          0.808           0.788   
                  RGCN                                 0.732           0.701   
                  Soft Medoid GDC                      0.772  \textit{0.769}   
                  Soft Median GDC                      0.791           0.789   
\textbf{Citeseer} Vanilla GCN                          0.700           0.675   
                  Vanilla GDC                          0.701           0.686   
                  SVD GCN                              0.639           0.627   
                  Jaccard GCN                          0.710           0.705   
                  RGCN                                 0.641           0.624   
                  Soft Medoid GDC                      0.706           0.704   
                  Soft Median GDC                      0.710           0.707   
\textbf{arXiv}    Vanilla GCN                          0.647           0.464   
                  Vanilla GDC                          0.622           0.617   
                  Soft Medoid GDC                      0.593           0.585   
\textbf{Products} Vanilla GCN                          0.712           0.661   
                  Vanilla GDC                          0.706           0.700   

\textbf{Attack}                                       \textbf{greedy FGSM}  \
Frac. edges \(\boldsymbol{\epsilon}\)      0.1   0.25                 0.01   
                  \textbf{Architecture}                                      
\textbf{Cora ML}  Vanilla GCN            0.732  0.658                0.786   
                  Vanilla GDC            0.762  0.712                0.795   
                  SVD GCN                0.760  0.722                0.757   
                  Jaccard GCN            0.768  0.737 

In [39]:
print(df_piv\
    .sort_index(level=1, ascending=False, sort_remaining=False)\
    .sort_index(level=0, ascending=True, sort_remaining=True)\
    .to_latex(
        float_format='%.2f',
        caption=r'Perturbed accuracy for the proposed attacks (see Sections~\ref{sec:attackkdd}-\ref{sec:prbcd}) and baselines on all datasets (see Table~\ref{tab:datasets}). \(\epsilon\) denotes the fraction of edges perturbed (relative to the clean graph). The last column contains the clean accuracy. As this a work-in-progress report, the experiments for the defenses on the large datasets are due and on Products we did not optimize the hyperparameters for GANG. For each architecture we italicize the strongest attack where \(\epsilon=0.05\), underline where \(\epsilon=0.1\), and embolden where \(\epsilon=0.25\). From an attack perspective, a lower perturbed accuracy is better. We rerun the experiments with three different seeds. For OGB we use the provided data splits and otherwise we use random split with 20 nodes per class.', 
        label='tab:global',
        escape=False,
        multirow=True,
        multicolumn=True,
        column_format='llccccccccccccccccccccccccc'
    )
)

\begin{table}
\centering
\caption{Perturbed accuracy for the proposed attacks (see Sections~\ref{sec:attackkdd}-\ref{sec:prbcd}) and baselines on all datasets (see Table~\ref{tab:datasets}). \(\epsilon\) denotes the fraction of edges perturbed (relative to the clean graph). The last column contains the clean accuracy. As this a work-in-progress report, the experiments for the defenses on the large datasets are due and on Products we did not optimize the hyperparameters for GANG. For each architecture we italicize the strongest attack where \(\epsilon=0.05\), underline where \(\epsilon=0.1\), and embolden where \(\epsilon=0.25\). From an attack perspective, a lower perturbed accuracy is better. We rerun the experiments with three different seeds. For OGB we use the provided data splits and otherwise we use random split with 20 nodes per class.}
\label{tab:global}
\begin{tabular}{llccccccccccccccccccccccccc}
\toprule
                  & \textbf{Attack} & \multicolumn{4}{l}{\textbf{DICE}}

# Datasets

In [15]:
def value_to_storage(val: float, decimals: int = 2):
    if val / 1e3 < 1:
        return f'{val:.{decimals}f} B'
    if val / 1e6 < 1:
        return f'{val / 1e3:.{decimals}f} kB'
    if val / 1e9 < 1:
        return f'{val / 1e6:.{decimals}f} MB'
    if val / 1e12 < 1:
        return f'{val / 1e9:.{decimals}f} GB'
    if val / 1e15 < 1:
        return f'{val / 1e12:.{decimals}f} TB'
    if val / 1e18 < 1:
        return f'{val / 1e15:.{decimals}f} PB'
    raise ValueError(f'{val} is too big for Peta!!!')

In [17]:
from ogb.nodeproppred import PygNodePropPredDataset

from rgnn_at_scale.data import load_dataset

dataset_df = []
for dataset in tqdm.tqdm(['cora_ml', 'citeseer', 'pubmed', 'ogbn-arxiv', 'ogbn-products', 'ogbn-papers100M']):
    if dataset.startswith('ogbn'):
        pyg_dataset = PygNodePropPredDataset(root='/nfs/staff-ssd/geisler/dontrobme/datasets', name=dataset)
        nnodes = pyg_dataset[0].x.shape[0]
        nedges = pyg_dataset[0].edge_index.shape[1]
        nfeatures = pyg_dataset[0].x.shape[1]
    else:
        graph = load_dataset(dataset, '/nfs/staff-ssd/geisler/dontrobme/datasets')
        #graph = load_and_standardize(dataset)
        nnodes = graph.adj_matrix.shape[0]
        nedges = graph.adj_matrix.nnz
        nfeatures = graph.attr_matrix.shape[1]
    dataset_df.append({
            r'\textbf{Dataset}': dataset_map[dataset].replace(r'rotatebox{90}', r'textbf'),
            r'\textbf{\#Nodes $n$}': f'{nnodes:,}',
            r'\textbf{\#Edges $e$}': f'{nedges:,}',
            r'\textbf{\#Features $d$}': f'{nfeatures:,}',
            r'\textbf{\#Possible edges}': f'{nnodes ** 2:.3E}',
            r'\textbf{Size (dense)}': value_to_storage(4 * nnodes ** 2),
            r'\textbf{Size (sparse)}': value_to_storage(2*8*nedges + 4*nedges),
        })
dataset_df = pd.DataFrame(dataset_df).set_index(r'\textbf{Dataset}')
dataset_df

100%|██████████| 6/6 [11:48<00:00, 118.08s/it]


,\textbf{\#Nodes $n$},\textbf{\#Edges $e$},\textbf{\#Features $d$},\textbf{\#Possible edges},\textbf{Size (dense)},\textbf{Size (sparse)}
\textbf{Dataset},,,,,,
\textbf{Cora ML},"2,995","8,416","2,879",8.970E+06,35.88 MB,168.32 kB
\textbf{Citeseer},"3,312","4,715","3,703",1.097E+07,43.88 MB,94.30 kB
\textbf{PubMed},"19,717","88,648",500,3.888E+08,1.56 GB,1.77 MB
\textbf{arXiv},"169,343","1,166,243",128,2.868E+10,114.71 GB,23.32 MB
\textbf{Products},"2,449,029","123,718,280",100,5.998E+12,23.99 TB,2.47 GB
\textbf{Papers 100M},"111,059,956","1,615,685,872",128,1.233E+16,49.34 PB,32.31 GB


In [18]:
print(dataset_df
    #.sort_index()
    .to_latex(
        float_format='%.2f',
        caption='Statistics of the used datasets. For the dense adjacency matrix we assume that each elements is represented by 4 bytes. In the sparse case we use two 8 byte integer pointers and a 4 bytes float value.', 
        label='tab:datasets',
        escape=False
    )
)

\begin{table}
\centering
\caption{Statistics of the used datasets. For the dense adjacency matrix we assume that each elements is represented by 4 bytes. In the sparse case we use two 8 byte integer pointers and a 4 bytes float value.}
\label{tab:datasets}
\begin{tabular}{lllllll}
\toprule
{} & \textbf{\#Nodes $n$} & \textbf{\#Edges $e$} & \textbf{\#Features $d$} & \textbf{\#Possible edges} & \textbf{Size (dense)} & \textbf{Size (sparse)} \\
\textbf{Dataset}     &                      &                      &                         &                           &                       &                        \\
\midrule
\textbf{Cora ML}     &                2,995 &                8,416 &                   2,879 &                 8.970E+06 &              35.88 MB &              168.32 kB \\
\textbf{Citeseer}    &                3,312 &                4,715 &                   3,703 &                 1.097E+07 &              43.88 MB &               94.30 kB \\
\textbf{PubMed}      &    